In [ ]:
!pip install pyngrok -q
!pip install streamlit -q
!pip install --upgrade plotly -q

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

In [ ]:
%%writefile preprocessing.py
import re

def correct_spellings(phrase):
  corrections = {
    'pricethe': 'price the',
    'loudnessand': 'loudness and',
    'muffeled': 'muffled',
    'expidite': 'expedite',
    'suerb': 'superb',
    'eeplaces': 'ear pieces',
    'exilent': 'excellent',
    'worthable': 'worth able',
    'soundaverage': 'sound average',
    'bukd': 'build',
    'breliant': 'brilliant',
    'dvsvinyl': 'dvd vinyl',
    'qudoes': 'kudos',
    'extarnal': 'external',
    'heaten': 'heats',
    'iseent': 'is not',
    'worth the prize': 'worth the price',
    "laptop's": 'laptop',
    "laptop’s": 'laptop',
    "aslo": "also",
    "qulity": "quality",
    "qaulity": "quality",
    "sable": "cable" 
  }
  for k, v in corrections.items():
    phrase = phrase.replace(f"{k}", v)
  return phrase

def undo_contractions(phrase):
    # specific
    phrase = re.sub(r"won[\'’]t", "will not", phrase)
    phrase = re.sub(r"can[\'’]t", "can not", phrase)

    # general
    phrase = re.sub(r"n[\'’]t", " not", phrase)
    phrase = re.sub(r"[\'’]re", " are", phrase)
    phrase = re.sub(r"[\'’]s", " is", phrase)
    phrase = re.sub(r"[\'’]d", " would", phrase)
    phrase = re.sub(r"[\'’]ll", " will", phrase)
    phrase = re.sub(r"[\'’]t", " not", phrase)
    phrase = re.sub(r"[\'’]ve", " have", phrase)
    phrase = re.sub(r"[\'’]m", " am", phrase)
    return phrase

emoji_regex = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                  "]+", re.UNICODE)

def preprocess_reviews(reviews):
  reviews['text'] = reviews['title'] + ' . ' + reviews['review']

  reviews['text_cleaned'] = (reviews['text']
    .fillna('')
    .str.replace(r'([a-z]+)([A-Z])', r'\1 \2') # badProduct bad Product
    .str.lower()
    .str.replace(emoji_regex, '')
    .str.replace('\n', '.')
    .str.replace(r'\s*\.+\s*', '. ') # dots and spaces
    .str.replace(r'([\{\(\[\}\)\]])', r' \1 ') # spaces between parenthesis
    .str.replace(r'([:])', r' \1 ') # spaces between :
    .str.replace(r'(\d+\.?\d*)', r' \1 ') # spaces between numbers
    .apply(correct_spellings)
    .apply(undo_contractions)
  )

  return reviews

Writing preprocessing.py


In [ ]:
%%writefile aspects_extraction.py
import pandas as pd
import numpy as np

def has_vectors(doc):
  return np.all([token.has_vector for token in doc])

def extract_doc_aspects(doc):

    prod_pronouns = ['it','this','they','these']

    rule1_pairs = []
    rule2_pairs = []
    rule3_pairs = []
    rule4_pairs = []
    rule5_pairs = []
    rule6_pairs = []
    rule7_pairs = []

    for token in doc:
        if token.text == 'product':
          continue

        ## FIRST RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect
        ## RULE = M is child of A with a relationship of amod
        A = "999999"
        M = "999999"
        if token.dep_ == "amod" and not token.is_stop:
            M = token.text
            A = token.head.text

            # add adverbial modifier of adjective (e.g. 'most comfortable headphones')
            M_children = token.children
            for child_m in M_children:
                if(child_m.dep_ == "advmod"):
                    M_hash = child_m.text
                    M = M_hash + " " + M
                    break

            # negation in adjective, the "no" keyword is a 'det' of the noun (e.g. no interesting characters)
            A_children = token.head.children
            for child_a in A_children:
                if(child_a.dep_ == "det" and child_a.text == 'no'):
                    neg_prefix = 'not'
                    M = neg_prefix + " " + M
                    break

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict1 = {"noun" : A, "adj" : M, "rule" : 1}
            rule1_pairs.append(dict1)


        # # SECOND RULE OF DEPENDANCY PARSE -
        # # M - Sentiment modifier || A - Aspect
        # Direct Object - A is a child of something with relationship of nsubj, while
        # M is a child of the same something with relationship of dobj
        # Assumption - A verb will have only one NSUBJ and DOBJ
        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text

            if((child.dep_ == "dobj" and child.pos_ == "ADJ") and not child.is_stop):
                M = child.text

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict2 = {"noun" : A, "adj" : M, "rule" : 2}
            rule2_pairs.append(dict2)


        ## THIRD RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect
        ## Adjectival Complement - A is a child of something with relationship of nsubj, while
        ## M is a child of the same something with relationship of acomp
        ## Assumption - A verb will have only one NSUBJ and DOBJ
        ## "The sound of the speakers would be better. The sound of the speakers could be better" - handled using AUX dependency

        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text

            if(child.dep_ == "acomp" and not child.is_stop):
                M = child.text

            # example - 'this could have been better' -> (this, not better)
            if(child.dep_ == "aux" and child.tag_ == "MD"):
                neg_prefix = "not"
                add_neg_pfx = True

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict3 = {"noun" : A, "adj" : M, "rule" : 3}
            rule3_pairs.append(dict3)


        ## FOURTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect

        #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
        # M is a child of the same something with relationship of advmod

        #Assumption - A verb will have only one NSUBJ and DOBJ

        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if((child.dep_ == "nsubjpass" or child.dep_ == "nsubj") and not child.is_stop):
                A = child.text

            if(child.dep_ == "advmod" and not child.is_stop):
                M = child.text
                M_children = child.children
                for child_m in M_children:
                    if(child_m.dep_ == "advmod"):
                        M_hash = child_m.text
                        M = M_hash + " " + child.text
                        break

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict4 = {"noun" : A, "adj" : M, "rule" : 4}
            rule4_pairs.append(dict4)

        ## FIFTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect

        #Complement of a copular verb - A is a child of M with relationship of nsubj, while
        # M has a child with relationship of cop

        #Assumption - A verb will have only one NSUBJ and DOBJ

        children = token.children
        A = "999999"
        buf_var = "999999"
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text

            if(child.dep_ == "cop" and not child.is_stop):
                buf_var = child.text

        if(A != "999999" and buf_var != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict5 = {"noun" : A, "adj" : token.text, "rule" : 5}
            rule5_pairs.append(dict5)


        ## SIXTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect
        ## Example - "It ok", "ok" is INTJ (interjections like bravo, great etc)

        children = token.children
        A = "999999"
        M = "999999"
        if(token.pos_ == "INTJ" and not token.is_stop):
            for child in children :
                if(child.dep_ == "nsubj" and not child.is_stop):
                    A = child.text
                    M = token.text

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict6 = {"noun" : A, "adj" : M, "rule" : 6}
            rule6_pairs.append(dict6)

        ## SEVENTH RULE OF DEPENDANCY PARSE -
        ## M - Sentiment modifier || A - Aspect
        ## ATTR - link between a verb like 'be/seem/appear' and its complement
        ## Example: 'this is garbage' -> (this, garbage)

        children = token.children
        A = "999999"
        M = "999999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.text

            if((child.dep_ == "attr") and not child.is_stop):
                M = child.text

            if(child.dep_ == "neg"):
                neg_prefix = child.text
                add_neg_pfx = True

        if (add_neg_pfx and M != "999999"):
            M = neg_prefix + " " + M

        if(A != "999999" and M != "999999"):
            if A in prod_pronouns :
                A = "product"
            dict7 = {"noun" : A, "adj" : M, "rule" : 7}
            rule7_pairs.append(dict7)

    aspects = []

    aspects = rule1_pairs + rule2_pairs + rule3_pairs +rule4_pairs +rule5_pairs + rule6_pairs + rule7_pairs

    return aspects

def extract_aspects(nlp, reviews):
  aspects = []

  data = ([
    (x[1], x[0]) for x in reviews['text_cleaned'].reset_index().to_numpy()
  ])

  for doc, review_id in nlp.pipe(data, as_tuples=True):
    doc_aspects = extract_doc_aspects(doc)
    doc_aspects = [
        [review_id, aspect['noun'], aspect['adj'], aspect['rule']] 
        for aspect in doc_aspects if not aspect['noun'].lower().startswith('product')
    ]
    # filter aspects with out of vocubalary nouns
    doc_aspects = [
        doc_aspect for doc_aspect in doc_aspects 
        if has_vectors(nlp(doc_aspect[1]))
    ]
    aspects.extend(doc_aspects)

  aspects = pd.DataFrame(aspects, columns=['review_id', 'aspect', 'opinion', 'rule'])

  return aspects

Writing aspects_extraction.py


In [ ]:
%%writefile clustering.py
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def cluster_aspect_terms(nlp, aspects):

  aspect_terms = sorted(list(set(aspects['aspect'].values)))

  aspect_terms_sizes = aspects.groupby('aspect').size().sort_index().values

  aspect_terms_vectors = [doc.vector for doc in nlp.pipe(aspect_terms)]

  clusterer = AgglomerativeClustering(n_clusters=None,
                                      affinity='cosine',
                                      linkage='average',
                                      distance_threshold=0.2)

  clusterer.fit(aspect_terms_vectors)

  term_replacements = {}

  for cluster in range(clusterer.n_clusters_):

    idxs = np.nonzero(clusterer.labels_ == cluster)[0]

    terms = [t for i, t in enumerate(aspect_terms) if i in idxs]

    sizes = aspect_terms_sizes[idxs]
    
    main_term = terms[np.argmax(sizes)]

    for term in terms:
      term_replacements[term] = main_term

  return term_replacements

Writing clustering.py


In [ ]:
%%writefile app.py
import spacy
import pandas as pd
import streamlit as st
from preprocessing import preprocess_reviews
from aspects_extraction import extract_aspects
from clustering import cluster_aspect_terms
import plotly.express as px
import matplotlib.pyplot as plt


defaultCsv = {
    'Serco USB Hub + Sound Card': 'reviews.csv',
    'Honey': 'reviews_honey.csv',
}

st.set_page_config(
    page_title="Actionble Insights From Reviews",
    layout="wide",

)

@st.cache
def load_reviews(uploaded_file=None, default_file=None):

  if default_file is not None:
    reviews = pd.read_csv(default_file)

  if uploaded_file is not None:
    reviews = pd.read_csv(uploaded_file)
  
  reviews = validate_reviews_dataframe(reviews)

  return preprocess_reviews(reviews)

def validate_reviews_dataframe(r):
  if 'title' not in r.columns:
    raise ValueError("column title is required")
  if 'review' not in r.columns:
    raise ValueError("column review is required")
  if 'rating' not in r.columns:
    raise ValueError("column rating is required")
  if r['title'].dtype != 'O':
    raise ValueError("column title must be string")
  if r['review'].dtype != 'O':
    raise ValueError("column review must be string")
  if r['rating'].dtype != 'float64':
    raise ValueError("column rating must be float")
  r = r.dropna()
  if ((r['rating'] < 0) & (r['rating'] > 5)).any():
    raise ValueError("values in column rating must be between 0 and 5")
  return r

@st.cache(allow_output_mutation=True, suppress_st_warning=True)
def load_model():
  return spacy.load("en_core_web_lg")

@st.cache(allow_output_mutation=True, suppress_st_warning=True)
def get_aspects(reviews):
  nlp = load_model()
  return extract_aspects(nlp, reviews)

@st.cache(allow_output_mutation=True, suppress_st_warning=True)
def cluster_aspects(aspects):
  nlp = load_model()
  replacements = cluster_aspect_terms(nlp, aspects)
  aspects['aspect'] = aspects['aspect'].map(replacements)
  return aspects

def get_aspects_with_ratings(aspects, reviews):
  aspect_with_ratings = pd.merge(aspects,
  reviews[['rating']],
  left_on='review_id', 
  right_index=True)
  aspect_with_ratings['review_sentiment'] = pd.cut(aspect_with_ratings['rating'], 
        bins=[0, 3, 4, 5], 
        right=True,
        labels=['Negative', 'Neutral', 'Positive']
  )
  return aspect_with_ratings

def get_aspect_treemap(aspects):
  treemap = px.treemap(aspects.groupby(['aspect', 'opinion']).size().reset_index(),
      path=[px.Constant('Aspects'), 'aspect', 'opinion'],
      values=0,
  )
  treemap.update_layout(margin = dict(t=0, l=0, r=0, b=0))
  return treemap

def plot_pain_points(aspect_with_ratings):
  pain_points = (aspect_with_ratings
    .query('review_sentiment == "Negative"')
    .groupby('aspect')
    .size()
    .sort_values(ascending=False)[:10]
  )
  fig = px.bar(pain_points)
  fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
  fig.update_traces(marker_color='red', showlegend=False)
  return fig

def plot_gain_points(aspect_with_ratings):
  gain_points = (aspect_with_ratings
    .query('review_sentiment == "Positive"')
    .groupby('aspect')
    .size()
    .sort_values(ascending=False)[:10]
  )
  fig = px.bar(gain_points)
  fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
  fig.update_traces(marker_color='green', showlegend=False)
  return fig

def plot_sentiment_by_aspect(aspect_with_ratings, top=15):
  pivot = pd.crosstab(
    index=aspect_with_ratings['aspect'],
      columns=aspect_with_ratings['review_sentiment'],
      margins=True,
  ).sort_values(by='All', ascending=False).iloc[1:, :-1]

  fig = px.bar(pivot[:top], barmode='group', color_discrete_map={
      'Positive': 'green', 
      'Negative': 'red',
      'Neutral': 'blue',
  })
  fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
  return fig


st.write("## Actionble Insights From Reviews")

st.write("""
Key to building a successfull product is understanding what users want and what users don't want.

This insight can be useful in serveral ways.

1. Designing product that users actually want.
2. Fixing defects in product or addressing users pain points.
3. Staying ahead of the competition.

There are millions of reviews that people leave on sites like amazon, tripadvisor etc. 
To gain insights from this data, you could either read all the reviews one by one or 
let machine analyze these reviews and find main topics that user care about.
""")

st.write("## Extracting Aspect Opinion Pairs")
st.write("""
Let's say the customer wrote, `The material of the shirt is not soft`. 
Here `material` is the `aspect` of shirt and `not soft` is the users `opinion`
about this aspect. The analyzer finds aspect opinion pairs from the reviews.
""")

st.write("### Customer Reviews")
st.write("""
Dataframe containing reviews of the customer. Title, review, and rating columns are required
""")

st.sidebar.title("Select Reviews File")

default_file = st.sidebar.selectbox(
    "Choose Sample File", 
    defaultCsv.keys(),
)
if default_file is not None:
  default_file = defaultCsv[default_file]

st.sidebar.write("<div style='text-align:center'>or</div>",  unsafe_allow_html=True)


uploaded_file = st.sidebar.file_uploader(
    'Choose a CSV File',
    type='csv',
)
st.sidebar.write("CSV with title(string), review(string) and ratings(float 0-5) columns")

try:
  reviews = load_reviews(uploaded_file, default_file)
  st.write(reviews)

  aspects = get_aspects(reviews)
  aspects = cluster_aspects(aspects)
  aspects_with_ratings = get_aspects_with_ratings(aspects, reviews)

  st.write("### Extracted Aspect Opinion Pairs")
  st.write("""
  Treemap of aspect opinion pairs extracted from reviews, treemap
  is sized according to number of reviews.
  """)
  st.plotly_chart(get_aspect_treemap(aspects), use_container_width=True)


  st.write("### Pain Points And Gain Points")
  col1, col2 = st.columns(2)

  with col1:
    st.write('Top Pain Points (by number of -ve reviews)')
    st.plotly_chart(plot_pain_points(aspects_with_ratings), use_container_width=True)

  with col2:
    st.write('Top Gain Points (by number of +ve reviews)')
    st.plotly_chart(plot_gain_points(aspects_with_ratings), use_container_width=True)

  st.write("### Sentiment for each aspect")
  st.write('(0-3 Negative) (4 Neutral) (5 Positive)')
  st.plotly_chart(plot_sentiment_by_aspect(aspects_with_ratings), use_container_width=True)
except ValueError as e:
  st.error(e)

Overwriting app.py


In [ ]:
!mkdir .streamlit

mkdir: cannot create directory ‘.streamlit’: File exists


In [ ]:
%%writefile .streamlit/config.toml
[theme]
base="light"

Overwriting .streamlit/config.toml


In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
public_url = ngrok.connect(8081)
print(public_url)

NgrokTunnel: "http://96c6-34-86-231-222.ngrok.io" -> "http://localhost:8081"


In [ ]:
!streamlit run app.py --server.port=8081 > /dev/null

2021-11-30 11:22:48.722 NumExpr defaulting to 2 threads.
